In [22]:
import pandas as pd
from parsel import Selector
import time
import random
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service



In [23]:

from credentials import credentials



In [24]:
#credentials to sign in
email_id = credentials['email_id']
password_0 = credentials['password']


In [25]:



#Initialize Chrome webdriver
driver = webdriver.Chrome()

def validate_field(field):
    if field:
        pass
    else:
        field = "no results"
    return field
#Open LinkedIn website
driver.get('https://www.linkedin.com')

#Locate and input email/username
username = driver.find_element(By.ID, 'session_key')
username.send_keys(email_id)

#Add a short delay (optional)
sleep(0.5)

#Locate and input password
password = driver.find_element(By.ID, 'session_password')
password.send_keys(password_0)

#Locate and click the sign-in button
sign_in_button = driver.find_element(By.XPATH, '//*[@type="submit"]')
sign_in_button.click()

#Add a longer delay to allow time for the sign-in process
sleep(12)



In [26]:
#Search for the profile we want to crawl

#Locate the search bar element
search_field = driver.find_element(By.XPATH, '//*[@id="global-nav-typeahead"]/input')
#Input the search query to the search bar
full_name = input("Enter the name you wish to look up: ")
search_field.send_keys(f'{full_name}')
#Search
search_field.send_keys(Keys.RETURN)


Enter the name you wish to look up:  daksh patel


In [28]:
#find people button by xpath
people = driver.find_element(By.XPATH,'//*[@id="search-reusables__filters-bar"]/ul/li[1]/button')
#button click
people = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[1]/button')))

people.click()
sleep(2)

In [29]:
#getting URLs of the profiles
profiles = driver.find_elements(By.CLASS_NAME, 'app-aware-link')
all_URL = []
for profile in profiles:
    profile_ID = profile.get_attribute('href')
    profile_URL = profile_ID
    if profile_URL not in all_URL:
        all_URL.append(profile_URL)

print('done')

done


In [30]:
main_profile_URL = []
for items in all_URL:
    if len(items)>40:
        main_profile_URL.append(items)

In [66]:
profiledata =[]
for link in main_profile_URL:
    driver.get(link)
    time.sleep(random.uniform(2.5,4.9))
    
    sel = Selector(text=driver.page_source)
    name = sel.xpath('//*[starts-with(@class,"text-heading-xlarge inline t-24 v-align-middle break-words")]/text()').extract_first()
    if name:
        name = name.strip()
    job_title = sel.xpath('//*[starts-with(@class,"text-body-medium break-words")]/text()').extract_first()

    if job_title:
        job_title = job_title.strip()
    
    connections = driver.find_element(By.XPATH,'//p[contains(@class, "wEMnHRWZkPesuVSxhIljYnMUSvZWxU")]/span[@aria-hidden="true"]').text
    if connections:
        connections = connections.strip()

    try:
        company = driver.find_element(By.XPATH,'//*[@id="profile-content"]/div/div[2]/div/div/main/section[1]/div[2]/div[2]/ul/li/button/span').text
    except:
        company = 'None'
    if company:
        company = company.strip()
    location = sel.xpath('//*[starts-with(@class, "text-body-small inline t-black--light break-words")]/text()').extract_first()
    if location:
        location.strip()
    about = sel.xpath('//*[@id="profile-content"]/div/div[2]/div/div/main/section[3]/div[3]/div/div/div/span[2]/text()').extract_first()
    if about:
        about = about.strip()

    name = validate_field(name)
    job_title = validate_field(job_title)
    company = validate_field(company)
    location = validate_field(location)
    about = validate_field(about)
   

    print('\n')
    print('Name: ' + name)
    print('Job Title: ' + job_title)
    print('Company: ' + company)
    print('Location: ' + location)
    print('About: ' + about)
    print('Connections: ' + connections)
    print('URL: ' + link)
    print('\n')

    data = {
        'URL': link,
        'Name': name,
        'Job Title': job_title,
        'Company': company,
        'Location':location,
        'About': about,
        'Connections': connections
        
    }
        
    profiledata.append(data)
    
profiledataframe = pd.DataFrame(profiledata)
driver.quit()



Name: Daksh Patel
Job Title: Business Development Manager
Company: Logix Built Solutions Pvt Ltd
Location: 
          Surat, Gujarat, India
        
About: Hello, I'm Daksh Patel, a results-oriented Business Development Manager in the IT/Software Service industry. With expertise in driving client acquisition and market analysis, I consistently surpass targets. My strength lies in cultivating robust client relationships and thriving in collaborative tech environments. Let's connect for innovative solutions and business growth
Connections: 7 followers
URL: https://www.linkedin.com/in/daksh-patel-7b77322ab?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEquSPsBTO6mjTCnmnb6TmzIHA2kASmrTCo




Name: Daksh Patel
Job Title: Student at The Maharaja Sayajirao University of Baroda
Company: The Maharaja Sayajirao University of Baroda
Location: 
          Gujarat, India
        
About: no results
Connections: 125 followers
URL: https://www.linkedin.com/in/daksh-patel-5017021a7?miniProfileUrn=urn

In [67]:
profiledataframe.to_csv('profiledata.csv')

In [68]:
profiledataframe

,URL,Name,Job Title,Company,Location,About,Connections
0,https://www.linkedin.com/in/daksh-patel-7b7732...,Daksh Patel,Business Development Manager,Logix Built Solutions Pvt Ltd,"\n Surat, Gujarat, India\n","Hello, I'm Daksh Patel, a results-oriented Bus...",7 followers
1,https://www.linkedin.com/in/daksh-patel-501702...,Daksh Patel,Student at The Maharaja Sayajirao University o...,The Maharaja Sayajirao University of Baroda,"\n Gujarat, India\n",no results,125 followers
2,https://www.linkedin.com/in/daksh-patel-922ba5...,Daksh Patel,Data Scientist | GDSC Tech Lead,Google Developer Student Clubs,"\n Gandhinagar, Gujarat, India\n",Passionate about leveraging data to drive insi...,301 followers
3,https://www.linkedin.com/in/daksh-patel-?miniP...,Daksh Patel,Digital Product Designer,SandCup Design Studio,"\n Ahmedabad, Gujarat, India\n",Creating mindful and intuitive human-centered ...,"1,814 followers"
4,https://www.linkedin.com/in/daksh-patel-aa8699...,Daksh Patel,"""Passionate Innovator | Driving Positive Chang...",Roma Industries,"\n Ahmedabad, Gujarat, India\n",Experienced Director with a demonstrated histo...,83 followers
5,https://www.linkedin.com/in/daksh-patel-30581b...,Daksh Patel,JAVA DEVELOPER || CORE JAVA || Well Known ...,"LDRP Institute of Technology & Research, Gujar...","\n Gandhinagar, Gujarat, India\n",no results,175 followers
6,https://www.linkedin.com/in/daksh-patel-920785...,Daksh Patel,Associate software Engineer at Thomson Reuters,Thomson Reuters,"\n Ahmedabad, Gujarat, India\n",no results,935 followers
7,https://www.linkedin.com/in/-daksh-patel?miniP...,Daksh Patel,SDE @Jio ( DSA | JAVA | LLD | REACTJS | NODEJS...,Jio,"\n Ahmedabad, Gujarat, India\n",-Experienced Software Developer with a demonst...,"2,276 followers"
8,https://www.linkedin.com/in/daksh-p-0361b3121?...,Daksh P.,no results,None,"\n Gujarat, India\n",An engineer who has a vision to merge data sci...,499 followers
9,https://www.linkedin.com/in/daksh-patel-811a9b...,Daksh Patel,Director,Tirex Chargers,"\n Ahmedabad, Gujarat, India\n",no results,226 followers
